In [1]:
import numpy as np
import os
import glob
import pandas as pd
from statsmodels.sandbox.stats.multicomp import multipletests
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from subprocess import call

In [14]:
dir_pre = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/ASE/nano_merge"
cov = 200
output_path = f"{dir_pre}_signal_eqtl.csv"

In [15]:
files = glob.glob(f'{dir_pre}_haplotype_chr*_tmp.csv')
dfs = [pd.read_csv(file) for file in files]
df = pd.concat(dfs, ignore_index=True)
df = df.sort_values(by=['chrom', 'snp_pos_1base'])
df = df.reset_index(drop=True)
df['cov'] = df['A1_num']+df['A2_num']
df = df[df['cov'] >= cov]

In [16]:
# binomial distribution
def binomial_distribution_logaFC(ref_num, alt_num, bias):
    total_num = ref_num + alt_num
    p_value = stats.binomtest(ref_num, total_num, bias, alternative='two-sided').pvalue
    p_value = max(p_value, 1e-300)
    if ref_num == 0:
        beta = np.log2(alt_num + 1)  # 或者返回一个大值，表示替代等位基因表达显著高于参考等位基因
    elif alt_num == 0:
        beta = np.log2(1 / (ref_num + 1))  # 或者返回一个小值，表示参考等位基因表达显著高于替代等位基因
    else:
        beta = np.log2(ref_num / alt_num)
    return p_value,beta

def apply_simulate(row):
    return binomial_distribution_logaFC(row['A1_num'], row['A2_num'], row['MAF'])

In [17]:
results = df.apply(apply_simulate, axis=1, result_type='expand')
df[['p_value','beta']] = results

In [18]:
p_adjusted = multipletests(df["p_value"], method='fdr_bh')
df['fdr'] = p_adjusted[1]
print("pvalue小于0.05的个数：", len(df[df['p_value']<0.05]))
print("pvalue小于0.01的个数：", len(df[df['p_value']<0.01]))
print("fdr小于0.1的个数：", len(df[df['fdr']<0.1]))

pvalue小于0.05的个数： 3263
pvalue小于0.01的个数： 2591
fdr小于0.1的个数： 3189


In [65]:
result_df = df[df['p_value']<0.05]
result_df.to_csv(output_path, index=False)